Необходимые импорты

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random
import random
from random import randint
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Проверка сохранения файла

In [2]:
a = np.array([[1, 2, 3, 4, 5],[0, 2, 3, 3, 2]])

with open('/content/drive/My Drive/array.txt', 'w') as f:
    np.savetxt(f, a)

Функция, преобразующая строку из 64 чисел в массив 4 на 4 на 4

In [3]:
def line_to_array(pos):
    position = (([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]),
                ([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]),
                ([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]),
                ([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]))
    
    counter = 0

    for k in range(4):
        for i in range(4):
            for j in range(4):
                position[k][i][j] = pos[counter]
                counter += 1
                
    return position

Функция проверки победы

In [4]:
def check_win(pos):
    if 0 not in pos:
        return True

    position = line_to_array(pos)
    if ((position[0][0][0] == position[1][1][1] == position[2][2][2] == position[3][3][3] != 0) \
        or (position[3][0][0] == position[2][1][1] == position[1][2][2] == position[0][3][3] != 0) \
        or (position[0][0][3] == position[1][1][2] == position[2][2][1] == position[3][3][0] != 0) \
        or (position[0][3][0] == position[1][2][1] == position[2][1][2] == position[3][0][3] != 0)):
        return True

    for k in range(4):
        for i in range(4):
            for j in range(4):
                if ((position[k][0][0] == position[k][1][1] == position[k][2][2] == position[k][3][3] != 0) \
                    or (position[k][3][0] == position[k][2][1] == position[k][1][2] == position[k][0][3] != 0) \
                    or (position[k][0][j] == position[k][1][j] == position[k][2][j] == position[k][3][j] != 0) \
                    or (position[k][i] == [1, 1, 1, 1]) or (position[k][i] == [-1, -1, -1, -1]) \
                    or (position[0][i][j] == position[1][i][j] == position[2][i][j] == position[3][i][j] != 0) \
                    or (position[0][i][0] == position[1][i][1] == position[2][i][2] == position[3][i][3] != 0) \
                    or (position[3][i][0] == position[2][i][1] == position[1][i][2] == position[0][i][3] != 0) \
                    or (position[0][0][j] == position[1][1][j] == position[2][2][j] == position[3][3][j] != 0) \
                    or (position[3][0][j] == position[2][1][j] == position[1][2][j] == position[0][3][j] != 0)):
                    return True

    return False

In [5]:
# a = np.zeros(64, dtype=int)

In [6]:
# %%timeit
# check_win(a)

Константы обучения

In [7]:
mutation_chance = 0.1
mutation_coef = 0.05
mutation_parametr = 0.02
cross_chance = 0.9
cross_parametr = 0.2
population = 256
iterations = 20000
tournament_count = 4
evaluation_precise = 25

Класс игрока

In [8]:
class Player:
    """
    Играет в крестики нолики. Может выдавать ход по данной позиции.
    Имеет self.weights - матрица 64 на 64 целых чисел, и self.bias - строка 1 на 64 целых чис.
    """
    def __init__(self):
        """
        Инициализирует bias и weights случайными целыми числами.
        """
        self.bias = np.random.randint(-3, 3, 64)
        self.weights = np.random.randint(-10, 10, (64, 64))

    def mutation(self):
        """
        Изменяет некоторые случйно выбранные числа матрицы weights и строки bias на небольшое число.
        """
        for i in range(64):
            if randint(1, 100) <= 100 * mutation_parametr:
                self.bias[i] += randint(-2, 2)
                if self.bias[i] > 10:
                    self.bias[i] = 10
                if self.bias[i] < -10:
                    self.bias[i] = -10

            for j in range(64):
                if randint(1, 100) <= 100 * mutation_parametr:
                    self.weights[i][j] += randint(-7, 7)
                    if self.weights[i][j] > 100:
                        self.weights[i][j] = 100
                    if self.weights[i][j] < -100:
                        self.weights[i][j] = -100

    def predict(self, input_values):
        return np.dot(input_values, self.weights) + self.bias

In [1]:
def cross_old(parent1, parent2):
    """
    Пример плохого кроссинговера.
    arguments:
    parent1 и parent2 - родители. Класс Player.
    returns:
    child - ребенок. Класс Player.
    """
    child = Player()
    for i in range(64):
        if randint(0, 1) == 1:
            child.bias[i] = parent1.bias[i]
        else:
            child.bias[i] = parent2.bias[i]
        for j in range(64):
            if randint(0, 1) == 1:
                child.weights[i][j] = parent1.weights[i][j]
            else:
                child.weights[i][j] = parent2.weights[i][j]
    return child

In [10]:
def cross(parent1, parent2):
    """
    Кроссинговер.
    arguments:
    parent1 и parent2 - родители. Класс Player.
    returns:
    child - ребенок. Класс Player.
    """
    child = Player()
    for i in range(64):
        if randint(0, 1) == 1:
            child.bias[i] = parent1.bias[i]
        else:
            child.bias[i] = parent2.bias[i]
        if randint(0, 1) == 1:
            for j in range(64):
                child.weights[i][j] = parent1.weights[i][j]
        else:
            for j in range(64):
                child.weights[i][j] = parent2.weights[i][j]
    return child

In [11]:
def game(p1, p2):
    """
    Игра между двумя игроками.
    arguments:
    p1 - игрок за крестиков. Класс Player.
    р2 - игрок за ноликов. Класс Player.
    returns:
    p1 или p2- победитель. Класс Player.
    """
    position = np.zeros(64, dtype = int)
    
    while not check_win(position):

        # X - move
        res1 = p1.predict(position)
        while position[np.argmax(res1)] != 0:
            res1[np.argmax(res1)] = -99999
        position[np.argmax(res1)] = 1
        if check_win(position):
            return p1

        # O - move
        res2 = p2.predict((-1) * position)
        while position[np.argmax(res2)] != 0:
            res2[np.argmax(res2)] += -99999
        position[np.argmax(res2)] = -1
    
    return p2

In [12]:
def game_with_comments (p1, p2):
    """
    Та же game, только с комментариями.
    arguments:
    p1 - игрок за крестиков. Класс Player.
    р2 - игрок за ноликов. Класс Player.
    returns:
    p1 или p2- победитель. Класс Player.
    """
    position = np.zeros(64)
    
    while not check_win(position):
        res1 = p1.predict(position)
        while position[np.argmax(res1)] != 0:
            res1[np.argmax(res1)] = -999999
        position[np.argmax(res1)] = 1
        print('X - ', np.argmax(res1)) #######
        if check_win(position):
            # print(line_to_array(position))
            return p1

        res2 = p2.predict((-1) * position)
        while position[np.argmax(res2)] != 0:
            res2[np.argmax(res2)] += -999999

        position[np.argmax(res2)] = -1
        print('O - ' ,np.argmax(res2)) #######
    
    # print(line_to_array(position))
    return p2

In [13]:
def championship(participants):
    """
    arguments:
    participants - участники турнира. Массив экземпляров класса Player.
    returns:
    winners - победители первого раунда, массив длины половины длины participants экземпляров класса Player.
    """
    length = len(participants)
    winners = [Player()] * (length // 2)
    counter = 0
    place = 0
    while counter < length:
        winners[place] = game(participants[counter], participants[counter + 1])
        place += 1
        counter += 2
    return winners

In [14]:
def evaluate(gamer_):
    """
    Измерение качества данного игрока (процент побед против случайных ходов и среднее кол-во ходов).
    arguments:
    gamer_ - игрок на оценку. Класс Player.
    returns:
    wins * 2 - процент побед. Тип int
    round(moves / 50, 2) - среднее кол-во ходов. Тип float
    """
    wins = 0
    moves = 0
    for i in range(25):
        position = np.zeros(64)
    
        while not check_win(position):

            # X_random - move
            res1 = randint(0, 63)
            while position[res1] != 0:
                res1 = randint(0, 63)
            position[res1] = 1
            moves += 1
            if check_win(position):
                break

            # O - move
            res2 = gamer_.predict((-1) * position)
            while position[np.argmax(res2)] != 0:
                res2[np.argmax(res2)] += -99999
            position[np.argmax(res2)] = -1
            moves += 1
            if check_win(position):
                wins += 1
                break

        position = np.zeros(64)
        
        while not check_win(position):

            # X - move
            res1 = gamer_.predict(position)
            while position[np.argmax(res1)] != 0:
                res1[np.argmax(res1)] += -99999
            position[np.argmax(res1)] = 1
            moves += 1
            if check_win(position):
                wins += 1
                break

            # O_random - move
            res2 = randint(0, 63)
            while position[res2] != 0:
                res2 = randint(0, 63)
            position[res2] = -1
            moves += 1
            if check_win(position):
                break
    
    return wins * 2, round(moves / 50, 2)


In [15]:
def iteration(gamers):
    """
    Осуществляет одну итерацию обучения.
    arguments:
    gamers - массив экземпляров класса Player длинны population.
    returns:
    gamers2 - массив той же длины, готовый ко второй итерации. 
    """
    gamers2 = [0] * population
    gamers2[0:population // 2] = championship(gamers)
    count = population // 2
    while count < population:
        gamers2[count] = cross(gamers2[randint(0, population // 2 - 1)], gamers2[randint(0, population // 2 - 1)])
        count += 1
    
    for individ in gamers2:
        if randint(1, 100) < mutation_chance * 100:
            individ.mutation()
    
    return gamers2

In [16]:
def chose_best(players):
    """
    Вычисляет двух лучших игроков путем турнира на выбывание.
    arguments:
    players - массив игроков длинны population. Типы Player
    returns - массив из двух игроков. Тип Player
    """
    n = len(players)
    while n > 2:
        n = n // 2
        for i in range(n):
            players[i] = game(players[i], players[-1 - i - len(players) + 2 * n])
    return [players[0], players[1]]

In [17]:
def write_to_file(_player1, _player2):
    """
    Записывает в файл двух игроков.
    arguments:
    _player1 - первый Player.
    _player2 - второй Player.
    """
    with open('/content/drive/My Drive/better_player1_weights.txt', 'w') as f:
        np.savetxt(f, _player1.weights, fmt = '%3d')
    with open('/content/drive/My Drive/better_player1_bias.txt', 'w') as f:
        np.savetxt(f, _player1.bias, fmt = '%3d')

    with open('/content/drive/My Drive/better_player2_weights.txt', 'w') as f:
        np.savetxt(f, _player2.weights, fmt = '%3d')
    with open('/content/drive/My Drive/better_player2_bias.txt', 'w') as f:
        np.savetxt(f, _player2.bias, fmt = '%3d')

Собственно цикл обучения

In [ ]:
boys = []
for sosi in range(population):
    boys.append(Player())

winrate_for_plot = [0] * iterations
moves_for_plot = [0] * iterations

for i in range(iterations):
    if i % 1000 == 0:
        print("Iteration", i + 1, '...')
        best = chose_best(boys)
        game_with_comments(best[1], best[0])
        write_to_file(best[0], best[1])
        
    random.shuffle(boys)
    boys = iteration(boys)
    # quality = [0] * 4
    # for u in range(4):
    #     quality[u] = evaluate(boys[randint(0, population - 1)])
    #     winrate_for_plot[i] += quality[u][0]
    #     moves_for_plot[i] += quality[u][1]
    
    # winrate_for_plot[i] /= 4
    # moves_for_plot[i] /= 4
    # print("Winrate on iteration", i + 1, winrate_for_plot[i])
    # print("Moves on iteration", i + 1, round(moves_for_plot[i], 2))

Iteration 1 ...
X -  8
O -  14
X -  30
O -  3
X -  62
O -  33
X -  18
O -  17
X -  16
O -  57
X -  19
O -  13
X -  0
O -  26
X -  21
O -  9
X -  25
O -  49
X -  27
O -  23
X -  22
O -  34
X -  54
O -  12
X -  42
O -  28
X -  4
O -  58
X -  53
O -  60
X -  31
O -  56
X -  50
O -  35
X -  55
O -  44
Iteration 1001 ...
X -  19
O -  1
X -  4
O -  40
X -  47
O -  42
X -  22
O -  13
X -  25
O -  59
X -  6
O -  8
X -  36
O -  37
X -  21
O -  9
X -  45
O -  14
X -  2
O -  43
X -  61
O -  35
X -  28
Iteration 2001 ...
X -  8
O -  37
X -  10
O -  34
X -  46
O -  40
X -  11
O -  0
X -  58
O -  15
X -  6
O -  21
X -  62
O -  12
X -  50
O -  39
X -  44
O -  35
X -  19
O -  13
X -  14
O -  22
X -  28
O -  53
X -  26
O -  5
Iteration 3001 ...
X -  29
O -  7
X -  10
O -  53
X -  11
O -  31
X -  9
O -  46
X -  48
O -  38
X -  41
O -  54
X -  60
O -  61
X -  3
O -  21
X -  12
O -  45
X -  35
O -  19
X -  22
Iteration 4001 ...
X -  29
O -  22
X -  8
O -  34
X -  39
O -  49
X -  11
O -  42
X -  56
O -  38

In [ ]:
best = chose_best(boys)
game_with_comments(best[1], best[0])
print(best[1].weights)
print(best[0].weights)

In [ ]:
print(best[0].bias)

In [ ]:
a = Player()
with open('/content/drive/My Drive/_player1_weights.txt', 'r') as f:
    a.weights = np.loadtxt(f)
with open('/content/drive/My Drive/_player1_bias.txt', 'r') as f:
    a.bias = np.loadtxt(f)

game_with_comments(a, a)